In [5]:
import numpy as np
import struct


class MnistData:
    def __init__(self, train_image_path, train_label_path,
                 test_image_path, test_label_path):
        # 训练集和测试集的文件路径
        self.train_image_path = train_image_path
        self.train_label_path = train_label_path
        self.test_image_path = test_image_path
        self.test_label_path = test_label_path

        # 获取训练集和测试集数据
        # get_data()方法，参数为0获取训练集数据，参数为1获取测试集
        self.train_images, self.train_labels = self.get_data(0)
        self.test_images, self.test_labels = self.get_data(1)

    def get_data(self, data_type):
        if data_type == 0:  # 获取训练集数据
            image_path = self.train_image_path
            label_path = self.train_label_path
        else:  # 获取测试集数据
            image_path = self.test_image_path
            label_path = self.test_label_path

        with open(image_path, 'rb') as file1:
            image_file = file1.read()
        with open(label_path, 'rb') as file2:
            label_file = file2.read()

        label_index = 0
        image_index = 0
        labels = []
        images = []

        # 读取训练集图像数据文件的文件信息
        magic, num_of_datasets, rows, columns = struct.unpack_from('>IIII',
                                                                   image_file, image_index)
        image_index += struct.calcsize('>IIII')

        for i in range(num_of_datasets):
            # 读取784个unsigned byte，即一副图像的所有像素值
            temp = struct.unpack_from('>784B', image_file, image_index)
            # 将读取的像素数据转换成28*28的矩阵
            temp = np.reshape(temp, (28, 28))
            # 归一化处理
            temp = temp / 255
            images.append(temp)
            image_index += struct.calcsize('>784B')  # 每次增加784B

        # 跳过描述信息
        label_index += struct.calcsize('>II')
        labels = struct.unpack_from('>' + str(num_of_datasets) + 'B', label_file, label_index)

        images = np.array(images)
        images = images.reshape(num_of_datasets, 28, 28 )
        # one-hot
        labels = np.eye(10)[np.array(labels)]

        return images, labels

In [6]:
train_image_path = './data/train-images-idx3-ubyte'
train_label_path = './data/train-labels-idx1-ubyte'
test_image_path = './data/t10k-images-idx3-ubyte'
test_label_path = './data/t10k-labels-idx1-ubyte'

data = MnistData(train_image_path, train_label_path,
                 test_image_path, test_label_path)

print(type(data.train_images))
print(type(data.train_labels))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
print(type(data.train_images))
print(type(data.train_labels))

print(type(data.train_images[0]))
print(type(data.train_labels[0]))

print(data.train_images[0])
print(data.train_labels[0])

In [10]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(28, 28)),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(data.train_images, data.train_labels, epochs=5)

W0612 05:44:11.203917 139730924926720 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f14fc1af940>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0612 05:44:11.207954 139730924926720 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f14742a6668>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Epoch 1/5
60000/60000 [==============================] - 11s 190us/sample - loss: 0.4478 - accuracy: 0.8595
Epoch 2/5
60000/60000 [==============================] - 11s 191us/sample - loss: 0.1256 - accuracy: 0.9628
Epoch 3/5
60000/60000 [==============================] - 11s 191us/sample - loss: 0.0868 - accuracy: 0.9743
Epoch 4/5
60000/60000 [==============================] - 11s 187us/sample - loss: 0.0629 - accuracy: 0.9810
Epoch 5/5
60000/60000 [==============================] - 11s 184us/sample - loss: 0.0525 - accuracy: 0.9848


In [22]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data.train_images, data.train_labels, epochs=5)
# model.evaluate(x_test, y_test)

Epoch 1/5
60000/60000 [==============================] - 6s 98us/sample - loss: 0.2196 - accuracy: 0.9340
Epoch 2/5
60000/60000 [==============================] - 6s 95us/sample - loss: 0.0952 - accuracy: 0.9709
Epoch 3/5
60000/60000 [==============================] - 6s 93us/sample - loss: 0.0681 - accuracy: 0.9788
Epoch 4/5
60000/60000 [==============================] - 6s 94us/sample - loss: 0.0534 - accuracy: 0.9826
Epoch 5/5
60000/60000 [==============================] - 6s 96us/sample - loss: 0.0432 - accuracy: 0.9854


In [24]:
model.evaluate(data.test_images, data.test_labels)

10000/10000 [==============================] - 1s 69us/sample - loss: 0.0719 - accuracy: 0.9790


[0.07191091472541447, 0.979]

In [17]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print(type(x_train))
print(type(x_test))

# print(x_train[0])
print(y_train[0])

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
5
